#Import the libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
import cv2 
from google.colab.patches import cv2_imshow
from PIL import Image
import sys
import math
import random
from skimage.transform import rotate, AffineTransform
from skimage.util import random_noise

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torch.utils.data import TensorDataset, ConcatDataset
import random
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preparation

In [ ]:
# this cell is regarding the aumentation and normalization with the stats of data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2)),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    # transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = datasets.ImageFolder("/content/drive/MyDrive/projectDataset/animal90/animals", transform= transform)

In [ ]:
# this cell is spliting dataset to train and test
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
test_split = 0.2
dataset_size = len(dataset)
test_size = int(test_split * dataset_size)
train_size = dataset_size - test_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# this cell is spliting train to train and val
val_split = 0.2
train_size = len(train_dataset)
val_size = int(val_split * train_size)
train_size = train_size - val_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
# Define the data loaders for the train and test sets
train_loader = DataLoader(train_dataset, batch_size=128, shuffle= True,
                          drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle= True,
                        drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle= True,
                         drop_last=True)

In [ ]:
num_class = len(dataset.classes)

In [ ]:
print(num_class)

85


#Finetune resnet 18

In [ ]:
# Define the ResNet18 model
model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_class)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train phase
from tqdm import tqdm

# Define the number of epochs
num_epochs = 50
loss_train = []
loss_val = []
loss_test = []

accuracy_train = []
accuracy_val = []
accuracy_test = []

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss_train = 0.0
    correct_train = 0
    total_train = 0
    running_loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # loss
        running_loss_train += loss.item()
        # accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    accuracy = 100 * correct_train / total_train
    loss_train.append(running_loss_train/len(train_loader))
    accuracy_train.append(accuracy)
    print(f"Accuracy of the network on the train images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on validation
    model.eval()
    correct_val = 0
    total_val = 0
    running_loss_val = 0
    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data
            val_inputs = val_inputs.to(DEVICE)
            val_labels = val_labels.to(DEVICE)
            # loss
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_val += loss.item()
            # accuracy
            _, predicted_val = torch.max(val_outputs.data, 1)
            total_val += val_labels.size(0)
            correct_val += (predicted_val == val_labels).sum().item()
    accuracy = 100 * correct_val / total_val
    loss_val.append(running_loss_val/len(val_loader))        
    accuracy_val.append(accuracy)
    print(f"Accuracy of the network on the val images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on test
    model.eval()
    correct_test = 0
    total_test = 0
    running_loss_test = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_test += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    accuracy = 100 * correct_test / total_test
    loss_test.append(running_loss_test/len(test_loader)) 
    accuracy_test.append(accuracy)
    print(f"Accuracy of the network on the test images after {epoch + 1} epochs is: {accuracy}")
    print('------------------------')

  0%|          | 0/50 [00:00<?, ?it/s]

Accuracy of the network on the train images after 1 epochs is: 1.375
Accuracy of the network on the val images after 1 epochs is: 1.171875


  2%|▏         | 1/50 [21:03<17:11:30, 1263.08s/it]

Accuracy of the network on the test images after 1 epochs is: 1.171875
------------------------
Accuracy of the network on the train images after 2 epochs is: 2.5625
Accuracy of the network on the val images after 2 epochs is: 2.9947916666666665


  4%|▍         | 2/50 [42:16<16:55:31, 1269.41s/it]

Accuracy of the network on the test images after 2 epochs is: 3.22265625
------------------------
Accuracy of the network on the train images after 3 epochs is: 3.6875
Accuracy of the network on the val images after 3 epochs is: 3.90625


  6%|▌         | 3/50 [1:02:00<16:03:43, 1230.28s/it]

Accuracy of the network on the test images after 3 epochs is: 4.4921875
------------------------


In [ ]:
#save the path
model_save_name = 'resnet18.pt'
path = f"/content/drive/MyDrive/projectDataset/animal90/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
# plot train and val and test accuracy 
plt.figure(figsize=(12,10), dpi=80)

plt.subplot(2, 2, 1)
plt.plot(loss_train, label="Train")
plt.plot(loss_val, color='red', label="Validation")
plt.title('Loss function for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(loss_test)
plt.title('Loss function for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(2, 2, 3)
plt.plot(accuracy_train, label="Train")
plt.plot(accuracy_val, color='red', label="Validation")
plt.title('Accuracy for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(accuracy_test)
plt.title('Accuracy for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# save accuracy and loss
import pickle
with open("/content/drive/MyDrive/projectDataset/animal90/accuracy_resnet18.txt", "wb") as fp:
    pickle.dump(accuracy_train, fp)
    pickle.dump(accuracy_val, fp)
    pickle.dump(accuracy_test, fp)

with open("/content/drive/MyDrive/projectDataset/animal90/loss_resnet18.txt", "wb") as fp:
    pickle.dump(loss_train, fp)
    pickle.dump(loss_val, fp)
    pickle.dump(loss_test, fp)

#ShuffleNetV2

In [ ]:
# Define the ShuflleNetV2 model
model = torchvision.models.shufflenet_v2_x1_0(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 90)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(DEVICE)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train phase
from tqdm import tqdm

# Define the number of epochs
num_epochs = 50
loss_train = []
loss_val = []
loss_test = []

accuracy_train = []
accuracy_val = []
accuracy_test = []

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss_train = 0.0
    correct_train = 0
    total_train = 0
    running_loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # loss
        running_loss_train += loss.item()
        # accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    accuracy = 100 * correct_train / total_train
    loss_train.append(running_loss_train/len(train_loader))
    accuracy_train.append(accuracy)
    print(f"Accuracy of the network on the train images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on validation
    model.eval()
    correct_val = 0
    total_val = 0
    running_loss_val = 0
    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data
            val_inputs = val_inputs.to(DEVICE)
            val_labels = val_labels.to(DEVICE)
            # loss
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_val += loss.item()
            # accuracy
            _, predicted_val = torch.max(val_outputs.data, 1)
            total_val += val_labels.size(0)
            correct_val += (predicted_val == val_labels).sum().item()
    accuracy = 100 * correct_val / total_val
    loss_val.append(running_loss_val/len(val_loader))        
    accuracy_val.append(accuracy)
    print(f"Accuracy of the network on the val images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on test
    model.eval()
    correct_test = 0
    total_test = 0
    running_loss_test = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_test += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    accuracy = 100 * correct_test / total_test
    loss_test.append(running_loss_test/len(test_loader)) 
    accuracy_test.append(accuracy)
    print(f"Accuracy of the network on the test images after {epoch + 1} epochs is: {accuracy}")
    print('------------------------')

In [ ]:
#save the path
model_save_name = 'shufflenet_v2_x1_0.pt'
path = f"/content/drive/MyDrive/projectDataset/animal90/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
# plot train and val and test accuracy 
plt.figure(figsize=(12,10), dpi=80)

plt.subplot(2, 2, 1)
plt.plot(loss_train, label="Train")
plt.plot(loss_val, color='red', label="Validation")
plt.title('Loss function for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(loss_test)
plt.title('Loss function for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(2, 2, 3)
plt.plot(accuracy_train, label="Train")
plt.plot(accuracy_val, color='red', label="Validation")
plt.title('Accuracy for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(accuracy_test)
plt.title('Accuracy for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# save accuracy and loss
import pickle
with open("/content/drive/MyDrive/projectDataset/animal90/accuracy_shufflenet_v2_x1_0.txt", "wb") as fp:
    pickle.dump(accuracy_train, fp)
    pickle.dump(accuracy_val, fp)
    pickle.dump(accuracy_test, fp)

with open("/content/drive/MyDrive/projectDataset/animal90/loss_shufflenet_v2_x1_0.txt", "wb") as fp:
    pickle.dump(loss_train, fp)
    pickle.dump(loss_val, fp)
    pickle.dump(loss_test, fp)

#MobileNetV2

In [ ]:
# Define the ShuflleNetV2 model
model = torchvision.models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 90)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(DEVICE)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train phase
from tqdm import tqdm

# Define the number of epochs
num_epochs = 50
loss_train = []
loss_val = []
loss_test = []

accuracy_train = []
accuracy_val = []
accuracy_test = []

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss_train = 0.0
    correct_train = 0
    total_train = 0
    running_loss_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # loss
        running_loss_train += loss.item()
        # accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()
    accuracy = 100 * correct_train / total_train
    loss_train.append(running_loss_train/len(train_loader))
    accuracy_train.append(accuracy)
    print(f"Accuracy of the network on the train images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on validation
    model.eval()
    correct_val = 0
    total_val = 0
    running_loss_val = 0
    with torch.no_grad():
        for val_data in val_loader:
            val_inputs, val_labels = val_data
            val_inputs = val_inputs.to(DEVICE)
            val_labels = val_labels.to(DEVICE)
            # loss
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_val += loss.item()
            # accuracy
            _, predicted_val = torch.max(val_outputs.data, 1)
            total_val += val_labels.size(0)
            correct_val += (predicted_val == val_labels).sum().item()
    accuracy = 100 * correct_val / total_val
    loss_val.append(running_loss_val/len(val_loader))        
    accuracy_val.append(accuracy)
    print(f"Accuracy of the network on the val images after {epoch + 1} epochs is: {accuracy}")

    # Evaluate the model on test
    model.eval()
    correct_test = 0
    total_test = 0
    running_loss_test = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(val_outputs, val_labels)
            running_loss_test += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    accuracy = 100 * correct_test / total_test
    loss_test.append(running_loss_test/len(test_loader)) 
    accuracy_test.append(accuracy)
    print(f"Accuracy of the network on the test images after {epoch + 1} epochs is: {accuracy}")
    print('------------------------')

In [ ]:
#save the path
model_save_name = 'mobilenet_v2.pt'
path = f"/content/drive/MyDrive/projectDataset/animal90/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
# plot train and val and test accuracy 
plt.figure(figsize=(12,10), dpi=80)

plt.subplot(2, 2, 1)
plt.plot(loss_train, label="Train")
plt.plot(loss_val, color='red', label="Validation")
plt.title('Loss function for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(loss_test)
plt.title('Loss function for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(2, 2, 3)
plt.plot(accuracy_train, label="Train")
plt.plot(accuracy_val, color='red', label="Validation")
plt.title('Accuracy for the training data')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(accuracy_test)
plt.title('Accuracy for the test data')
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# save accuracy and loss
import pickle
with open("/content/drive/MyDrive/projectDataset/animal90/accuracy_mobilenet_v2.txt", "wb") as fp:
    pickle.dump(accuracy_train, fp)
    pickle.dump(accuracy_val, fp)
    pickle.dump(accuracy_test, fp)

with open("/content/drive/MyDrive/projectDataset/animal90/loss_mobilenet_v2.txt", "wb") as fp:
    pickle.dump(loss_train, fp)
    pickle.dump(loss_val, fp)
    pickle.dump(loss_test, fp)